# Explanation of Decoding:
**orderHash**: Unique identifier for each order.

**maker**: Address of the maker (the person who placed the order).

**taker**: Address of the taker (the person who filled the order).

**makerAssetId**: The asset ID that the maker is offering.

**takerAssetId**: The asset ID that the taker is offering.

**makerAmountFilled**: The amount of the maker's asset that was filled.

**takerAmountFilled**: The amount of the taker's asset that was filled.

**fee**: Any fees associated with the transaction.

**event**: The type of event (OrderFilled, OrdersMatched, etc.).

**takerOrderHash**: The order hash of the taker's order (if applicable).

**takerOrderMaker**: The address of the taker's order maker (if applicable).

### Token Registered Specific
**token0**, **token1**, **conditionId**: Related to TokenRegistered events, representing tokens involved and the condition ID.

In [12]:
import csv
from web3 import Web3
from eth_abi import decode
import pandas as pd
event_signatures = {
    'OrderFilled': 'OrderFilled(bytes32,address,address,uint256,uint256,uint256,uint256,uint256)',
    'OrdersMatched': 'OrdersMatched(bytes32,address,uint256,uint256,uint256,uint256)',
    'PositionsMerge': 'PositionsMerge(address,address,bytes32,bytes32,uint256[],uint256)',
    'TokenRegistered': 'TokenRegistered(uint256,uint256,bytes32)',
    'NewOperator': 'NewOperator(address,address)',
    'FeeCharged': 'FeeCharged(address,uint256,uint256)',
    'NewAdmin': 'NewAdmin(address,address)',
    'TransferSingle': 'TransferSingle(address,address,address,uint256,uint256)'
}

event_topic0 = {name: Web3.keccak(text=signature).hex() for name, signature in event_signatures.items()}
print(event_topic0)

{'OrderFilled': 'd0a08e8c493f9c94f29311604c9de1b4e8c8d4c06bd0c789af57f2d65bfec0f6', 'OrdersMatched': '63bf4d16b7fa898ef4c4b2b6d90fd201e9c56313b65638af6088d149d2ce956c', 'PositionsMerge': '6f13ca62553fcc2bcd2372180a43949c1e4cebba603901ede2f4e14f36b282ca', 'TokenRegistered': 'bc9a2432e8aeb48327246cddd6e872ef452812b4243c04e6bfb786a2cd8faf0d', 'NewOperator': 'f1e04d73c4304b5ff164f9d10c7473e2a1593b740674a6107975e2a7001c1e5c', 'FeeCharged': 'acffcc86834d0f1a64b0d5a675798deed6ff0bcfc2231edd3480e7288dba7ff4', 'NewAdmin': 'f9ffabca9c8276e99321725bcb43fb076a6c66a54b7f21c4e8146d8519b417dc'}


In [17]:

print('Reading logs...')

def decode_address(topic):
    return '0x' + topic[-40:]

def decode_uint256(data):
    return int(data, 16)

def decode_order_filled(log):
    decoded_event = {}
    # Topic 1
    decoded_event['orderHash'] = log['topic1']
    # Topic 2
    decoded_event['maker'] = decode_address(log['topic2'])
    # Topic 3
    decoded_event['taker'] = decode_address(log['topic3'])
    # Data Field:
    data = log['data']
    chunks = [data[i:i+64] for i in range(0, len(data), 64)]
    decoded_event['makerAssetId'] = decode_uint256(chunks[0])
    decoded_event['takerAssetId'] = decode_uint256(chunks[1])
    decoded_event['makerAmountFilled'] = decode_uint256(chunks[2])
    decoded_event['takerAmountFilled'] = decode_uint256(chunks[3])
    decoded_event['fee'] = decode_uint256(chunks[4]) if len(chunks) > 4 else 0
    return decoded_event

def decode_orders_matched(log):
    decoded_event = {}
    # Topic 1
    decoded_event['takerOrderHash'] = log['topic1']
    # Topic 2
    decoded_event['takerOrderMaker'] = decode_address(log['topic2'])
    # No Topic 3
    # Data Field:
    data = log['data']
    # Data is in 4 chunks. We make chunks to decode easier
    chunks = [data[i:i+64] for i in range(0, len(data), 64)]
    # Chunk 1
    decoded_event['makerAssetId'] = decode_uint256(chunks[0])
    # Chunk 2
    decoded_event['takerAssetId'] = decode_uint256(chunks[1])
    # Chunk 3
    decoded_event['makerAmountFilled'] = decode_uint256(chunks[2])
    # Chunk 4
    decoded_event['takerAmountFilled'] = decode_uint256(chunks[3])
    return decoded_event

def decode_positions_merged(log):
    decoded_event = {}
    # Indexed parameters
    # Topic 1
    decoded_event['stakeholder'] = decode_address(log['topic1'])
    # Topic 2
    decoded_event['parentCollectionId'] = log['topic2']
    # Topic 3
    decoded_event['conditionId'] = log['topic3']
    # Non-indexed parameters
    # Data Field:
    data = log['data']
    data_bytes = bytes.fromhex(data[2:])  # Skip '0x' prefix
    # Decode the parameters using eth_abi
    types = ['address', 'uint256[]', 'uint256']
    decoded_values = decode(types, data_bytes)
    decoded_event['collateralToken'] = decoded_values[0]
    decoded_event['partition'] = decoded_values[1]
    decoded_event['amount'] = decoded_values[2]
    return decoded_event

def decode_token_registered(log):
    decoded_event = {}
    # Indexed parameters
    # Topic 1
    decoded_event['token0'] = int(log['topic1'], 16)
    # Topic 2
    decoded_event['token1'] = int(log['topic2'], 16)
    # Topic 3
    decoded_event['conditionId'] = log['topic3']
    # No Data Field
    return decoded_event

def decode_new_operator(log):
    decoded_event = {}
    # Topic 1
    decoded_event['newOperatorAddress'] = decode_address(log['topic1'])
    # Topic 2
    decoded_event['admin'] = decode_address(log['topic2'])
    # No Topic 3
    
    # No Data Field
    return decoded_event

def decode_fee_charged(log):
    decoded_event = {}
    # Topic 1
    decoded_event['receiver'] = decode_address(log['topic1'])
    # No Topic 2
    
    # No Topic 3
    
    # Data Field:
    data = log['data']
    data_bytes = bytes.fromhex(data[2:])  # Skip '0x' prefix
    # Decode the parameters using eth_abi
    types = ['uint256', 'uint256']
    decoded_values = decode(types, data_bytes)
    decoded_event['tokenId'] = decoded_values[0]
    decoded_event['amount'] = decoded_values[1]
    return decoded_event

def decode_new_admin(log):
    decoded_event = {}
    # Topic 1
    decoded_event['newAdminAddress'] = decode_address(log['topic1'])
    # Topic 2
    decoded_event['admin'] = decode_address(log['topic2'])
    # No Topic 3
    
    # No Data Field
    return decoded_event


# Read log entries
total_log_entries = 0
decoded_events = []
non_decoded_events = []
non_decoded_events_log_index = [] 

for i in range(36, 61):
    log_entries = []

        
    with open(f'ctf_exchange/logs_{i}M.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            log_entries.append(row)


    for log in log_entries:
        topic0 = log['topic0']
        total_log_entries += 1
        if topic0 == '0x' + event_topic0['OrderFilled']:
            decoded_event = decode_order_filled(log)
            decoded_event['event'] = 'OrderFilled'
        elif topic0 == '0x' + event_topic0['OrdersMatched']:
            decoded_event = decode_orders_matched(log)
            decoded_event['event'] = 'OrdersMatched'
        elif topic0 == '0x' + event_topic0['PositionsMerge']:
            decoded_event = decode_positions_merged(log)
            decoded_event['event'] = 'PositionsMerge'
        elif topic0 == '0x' + event_topic0['TokenRegistered']:
            decoded_event = decode_token_registered(log)
            decoded_event['event'] = 'TokenRegistered'
        elif topic0 == '0x' + event_topic0['NewOperator']:
            decoded_event = decode_new_operator(log)
            decoded_event['event'] = 'NewOperator'
        elif topic0 == '0x' + event_topic0['FeeCharged']:
            decoded_event = decode_fee_charged(log)
            decoded_event['event'] = 'FeeCharged'
        elif topic0 == '0x' + event_topic0['NewAdmin']:
            decoded_event = decode_new_admin(log)
            decoded_event['event'] = 'NewAdmin'

        

        else:
            if (non_decoded_events.__contains__(topic0)):
                continue

            print(log)
            non_decoded_events.append(topic0) 
            continue
        
        decoded_events.append(decoded_event)
print('Decoded events:', len(decoded_events))

Reading logs...
{'blockNumber': '0x38444c1', 'txIndex': '0x2', 'logIndex': '0x2', 'removed': '0', 'topic0': '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', 'topic1': '0x000000000000000000000000726081113bc3a84da236cff45fadb8feb58b66fd', 'topic2': '0x0000000000000000000000004bfb41d5b3570defd03c39a9a4d8de6bd8b8982e', 'topic3': '', 'data': '0x000000000000000000000000000000000000000000000000000000002059670f'}
{'blockNumber': '0x38444c1', 'txIndex': '0x2', 'logIndex': '0x3', 'removed': '0', 'topic0': '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', 'topic1': '0x000000000000000000000000726081113bc3a84da236cff45fadb8feb58b66fd', 'topic2': '0x0000000000000000000000004bfb41d5b3570defd03c39a9a4d8de6bd8b8982e', 'topic3': '', 'data': '0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffbee60ba1'}
{'blockNumber': '0x38444c1', 'txIndex': '0x2', 'logIndex': '0x4', 'removed': '0', 'topic0': '0xc3d58168c5ae7397731d063d5bbf3d657854427343f4c083240f7aacaa2d0

In [18]:
print('Total Entries: ' + f'{total_log_entries}')
print('To Decode Events: ' + f'{len(non_decoded_events)}')

print(non_decoded_events_log_index)
print(non_decoded_events)

Total Entries: 6521052
To Decode Events: 7
[]
['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', '0xc3d58168c5ae7397731d063d5bbf3d657854427343f4c083240f7aacaa2d0f62', '0x4a39dc06d4c0dbc64b70af90fd698a233a518aa5d07e595d983b8c0526c8f7fb', '0x2e6bb91f8cbcda0c93623c54d0403a43514fabc40084ec96b6d5379a74786298', '0x4dfe1bbbcf077ddc3e01291eea2d5c70c2b422b415d95645b9adcfd678cb1d63', '0x18fe0464eb77016dc4e227eb0d690e4002756d82b442143bbfb874548952b5f2']


In [15]:
df = pd.DataFrame(decoded_events)
df.to_csv('decoded_events.csv', index=False)

In [16]:
log_entries = []

with open('ctf_exchange/logs_58M.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            log_entries.append(row)

for decode_logs in non_decoded_events:
    print(decode_logs)
    if (log_entries.__contains__(decode_logs)):
        print(log_entries.__getitem__(decode_logs))